In [1]:
import pandas as pd
import pickle
import networkx as nx
from Propagation.utils import ReachFinder

with open("../state_files/Filtered Releases.pickle", "rb") as f:
    releases_dict = pickle.load(f)

G = nx.read_gexf("../state_files/PyPi Network V4.gexf")

known_dates_dict = {lib: {ver: date for ver, date in version_dict.items()
                          if date is not None}
                    for lib, version_dict in releases_dict.items()}

In [10]:
data_dict = dict()
data_dict["first date"] = {lib: min(version_dict.values()) for lib, version_dict in known_dates_dict.items()
                           if version_dict}  # False sii version_dict está vacío.
data_dict["last date"] = {lib: max(version_dict.values()) for lib, version_dict in known_dates_dict.items()
                           if version_dict}  # False sii version_dict está vacío.
data_dict["updates"] = {lib: len(version_dict.values()) for lib, version_dict in releases_dict.items()}
data_df = pd.DataFrame.from_dict(data=data_dict)

In [2]:
finder = ReachFinder(G)
reach = finder.find_all_nodes()

Function called 162575 times.

: 

: 

In [14]:
data_df.sort_values("updates", ascending=False)

,first date,last date,updates
ccxt,2017-07-12 13:27:57,2022-10-27 17:24:14,4168
spanishconjugator,2020-08-14 14:58:17,2022-10-27 20:31:25,3318
pulumi,2018-06-12 15:29:33,2022-10-26 14:29:38,3316
kcli,2019-12-15 12:59:59,2022-10-27 12:33:46,2402
tfa-nightly,2019-04-22 23:44:08,2022-10-11 21:16:03,2320
...,...,...,...
fred-cli,2020-08-27 20:37:02,2020-08-27 20:37:02,1
frechetdist,2019-08-12 12:23:40,2019-08-12 12:23:40,1
freaddb,2022-06-19 03:39:26,2022-06-19 03:39:26,1
frc,2021-11-18 16:07:50,2021-11-18 16:07:50,1
